## CS230 Deep Learning  | Model Baseline 

Fall 2019: Sheun Aluko, Surya Narayanan, Vadim Piccini

###  Import Dependencies 

Note: util.py defines several utilities for loading and processing raw data, as well as helper funcionts such as the IOU accuracy metric 

In [2]:
import util as u
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Activation
import matplotlib.pyplot as plt
keras = tf.keras

Loaded util


### Load dataset 

In [3]:
x_train,y_train,x_val,y_val,x_test,y_test = u.data_load(f=1)


Loading data for term index: 8
Fraction of data that will be loaded=1

Loading train set
Removed 6 lesion(s) of 1995
On index: 100
On index: 200
On index: 300
On index: 400
On index: 500
On index: 600
On index: 700
On index: 800
On index: 900
On index: 1000
On index: 1100
On index: 1200
On index: 1300
On index: 1400
On index: 1500
On index: 1600
On index: 1700
On index: 1800
On index: 1900
Done

Loading val set
Removed 0 lesion(s) of 179
On index: 100
Done

Loading test set
Removed 1 lesion(s) of 203
On index: 100
On index: 200
Done



In [ ]:
np.save("x_t")

In [ ]:
x_test.shape

In [ ]:
#set params
batch_size=1
num_epochs=300
model_name="v0_" + str(num_epochs) + "e_b" + str(batch_size)

In [ ]:
Y_train[0,:]

In [ ]:
X_train.shape

In [ ]:
#create model
model = Sequential()
#add model layers
model.add(Conv2D(10, kernel_size=11, input_shape=(512, 512, 3)))
model.add(Activation('relu'))
model.add(Conv2D(10, kernel_size=11))
model.add(Activation('relu'))

#pool
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(10, kernel_size=11))
model.add(Activation('relu'))
model.add(Conv2D(10, kernel_size=11))
model.add(Activation('relu'))

#pool 
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(4, activation = None))


#model.add(Dense(4, activation = "sigmoid", kernel_initializer=keras.initializers.RandomNormal(mean=0,stddev=0.05)))

# https://towardsdatascience.com/building-a-convolutional-neural-network-cnn-in-keras-329fbbadc5f5

In [ ]:
#compile model using accuracy to measure model performance
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.999), loss='mean_squared_error',metrics=[IoU])

In [ ]:
model.summary()

In [ ]:
h = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=batch_size, epochs=num_epochs)

In [ ]:
train_loss = h.history['loss']
test_loss  = h.history['val_loss']
epoch_count = range(1, len(train_loss) + 1)

plt.plot(epoch_count, train_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0,0.2])
plt.show();


In [ ]:
plt.savefig("models/" + model_name + ".png")

In [ ]:
def get_results(x,y) : 
    pred = model.predict(x)
    return (pred, y, y-pred , np.mean( (y-pred)**2) ) 
    

In [ ]:
y_train_p, y_train, train_diff , train_loss = get_results(X_train,Y_train)

In [ ]:
y_test_p, y_test, test_diff , test_loss = get_results(X_test,Y_test)

In [ ]:
y_train_p

In [ ]:
y_train

In [ ]:
(y_train_p - y_train)#train_diff

In [ ]:
train_loss

In [ ]:
# serialize model to JSON
def save_model(model,name) : 
    model_json = model.to_json()
    model_name = "models/" + name
    with open(model_name + ".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(model_name + ".h5")
    print("Saved model to disk")

In [ ]:
save_model(model,model_name)